In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, metrics
import matplotlib.pyplot as plt
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [2]:
#constants
LABELS = ['label_1', 'label_2', 'label_3', 'label_4']
FEATURES = ['feature_'+ str(i) for i in range(1,769)]

In [3]:
trainDf = pd.read_csv("layer_11_train.csv")
trainDf.head()
trainDf.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,0.042487,0.068749,0.145547,-0.070646,-0.013539,0.003395,-0.041282,-0.028283,-0.106602,0.053686,...,-0.022102,-0.044743,-0.004380,0.049072,-0.028722,0.075717,30.498843,27.975107,0.799299,5.997125
std,0.048918,0.046354,0.065332,0.046671,0.027635,0.031248,0.026479,0.029632,0.070775,0.030945,...,0.053250,0.031361,0.025829,0.050536,0.032622,0.044879,17.328389,5.735913,0.400532,2.375567
min,-0.079594,-0.062608,-0.048545,-0.307243,-0.178347,-0.194771,-0.197551,-0.304828,-0.421257,-0.049723,...,-0.253255,-0.264549,-0.137827,-0.117697,-0.302399,-0.090777,1.000000,22.000000,0.000000,0.000000
25%,0.009979,0.037225,0.100677,-0.088834,-0.027810,-0.010617,-0.056682,-0.044344,-0.135110,0.033236,...,-0.042332,-0.056918,-0.018848,0.012599,-0.045226,0.045309,15.000000,25.000000,1.000000,6.000000
50%,0.024445,0.056119,0.123554,-0.057386,-0.011423,0.006173,-0.041501,-0.025805,-0.080715,0.045567,...,-0.007960,-0.037407,-0.004701,0.033121,-0.022919,0.064875,30.000000,27.000000,1.000000,6.000000
75%,0.058410,0.086358,0.173234,-0.039661,0.001544,0.021250,-0.026559,-0.009324,-0.057873,0.065670,...,0.012116,-0.024179,0.010218,0.072599,-0.006335,0.097642,46.000000,30.000000,1.000000,6.000000
max,0.274146,0.332288,0.454182,0.059362,0.196950,0.213127,0.124194,0.105714,0.192121,0.252320,...,0.209455,0.054555,0.215375,0.376414,0.125857,0.416291,60.000000,61.000000,1.000000,13.000000


In [4]:
validDf = pd.read_csv("layer_11_valid.csv")
validDf.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.025707,0.072842,0.137001,-0.093690,0.015632,0.019517,-0.089173,-0.037077,-0.116238,0.097865,...,-0.043002,-0.037699,0.042082,0.006500,-0.038191,0.116253,45,NaN,1,6
1,0.017289,0.053504,0.141158,-0.046288,-0.037644,0.018299,-0.090750,0.020565,-0.150711,0.042980,...,-0.039593,-0.055229,0.001704,-0.005439,-0.035530,0.060964,45,NaN,1,6
2,0.030724,0.108214,0.115952,-0.101992,-0.015330,0.032235,-0.000838,-0.031209,-0.037022,0.060745,...,-0.029340,-0.046902,0.010382,0.047438,-0.056359,0.062159,45,NaN,1,6
3,0.039265,0.075224,0.112761,-0.108313,-0.004113,0.040537,-0.060936,-0.052650,-0.063517,0.083030,...,-0.003430,-0.021894,0.020697,0.026693,-0.013656,0.083940,45,NaN,1,6
4,0.029869,0.037192,0.107736,-0.038029,-0.002430,-0.002113,-0.059072,-0.045483,-0.099818,0.050547,...,-0.031195,-0.034666,-0.010491,0.036413,-0.028152,0.122546,45,NaN,1,6


In [5]:
testDf = pd.read_csv("layer_11_test.csv")
testDf.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,-0.000848,0.078148,0.187810,-0.041648,-0.033607,-0.038199,-0.089919,0.024080,-0.201969,0.064583,...,-0.078486,-0.070084,-0.053011,0.008485,-0.052973,0.078296,45,NaN,1,6
1,0.103377,0.159788,0.212854,-0.233145,-0.057392,-0.096155,-0.019025,-0.038726,-0.259538,0.155536,...,-0.176942,-0.089102,-0.010966,0.136578,-0.107316,0.217518,45,NaN,1,6
2,0.071094,0.130005,0.135423,-0.129862,0.018247,0.078235,-0.041788,-0.063971,-0.130495,0.099881,...,-0.020031,-0.024434,-0.033826,0.041559,-0.024067,0.097203,45,NaN,1,6
3,0.011934,0.030001,0.134863,-0.049538,-0.004478,0.002138,-0.045417,-0.018220,-0.055388,0.045562,...,-0.013422,-0.024320,0.001298,-0.008532,-0.045537,0.050802,45,NaN,1,6
4,0.025101,0.058108,0.118522,-0.072302,-0.007582,-0.006370,-0.003366,-0.032282,-0.104052,0.062415,...,-0.013517,-0.069421,-0.027624,0.042998,-0.001015,0.104858,45,NaN,1,6


In [6]:
# Check for missing values
trainDf.isnull().sum()

feature_1        0
feature_2        0
feature_3        0
feature_4        0
feature_5        0
              ... 
feature_768      0
label_1          0
label_2        480
label_3          0
label_4          0
Length: 772, dtype: int64

Label_2 has 480 rows with missing values. Those will be neglected.

## Scaling

Scaling the data using the standard scaler. This is used to transform and scale features in this dataset to have a mean of 0 and a standard deviation of 1, making them suitable to train the machine learning models

Train and validation data for all 4 labels will be scaled and stored in a dictionary for future reference.

In [7]:
data_dict = {"label_1" : {},"label_2" : {},"label_3" : {},"label_4" : {}}
tr_df = trainDf
vl_df = validDf
tst_df = testDf

for label in LABELS:
    if (label == 'label_2'):
        tr_df = trainDf[trainDf['label_2'].notna()]
        vl_df = validDf[validDf['label_2'].notna()]
    scaler = StandardScaler()
    data_dict[label]['X_train'] = pd.DataFrame(scaler.fit_transform(tr_df.drop(LABELS, axis = 1)), columns = FEATURES)
    data_dict[label]['Y_train'] = tr_df[label]
    data_dict[label]['X_valid'] = pd.DataFrame(scaler.transform(vl_df.drop(LABELS, axis = 1)), columns = FEATURES)
    data_dict[label]['Y_valid'] = vl_df[label]
    data_dict['X_test'] = pd.DataFrame(scaler.transform(tst_df.drop(LABELS, axis = 1)), columns = FEATURES)

After obtaining the best parameter values for each label, a separate model will be trained for each label and the test labels will be predicted

In [10]:
classifier1 = svm.SVC(kernel='rbf', random_state = 42, C= 100, gamma= 'auto')
classifier1.fit(data_dict['label_1']['X_train'], data_dict['label_1']['Y_train'])

y_valid_pred_conf = classifier1.predict(data_dict['label_1']['X_valid'])
y_test_pred_conf = classifier1.predict(data_dict['X_test'])
label_1_valid_df = pd.DataFrame(y_valid_pred_conf, columns=['label_1'])
label_1_test_df = pd.DataFrame(y_test_pred_conf, columns=['label_1'])

label_1_valid_df.to_csv('layer_11_label_1_valid.csv', index=False)
label_1_test_df.to_csv('layer_11_label_1_test.csv', index=False)

In [11]:
classifier2 = svm.SVC(kernel='rbf', random_state = 42, C= 100, gamma= 'scale')
classifier2.fit(data_dict['label_2']['X_train'], data_dict['label_2']['Y_train'])

y_valid_pred_conf = classifier2.predict(data_dict['label_2']['X_valid'])
y_test_pred_conf = classifier2.predict(data_dict['X_test'])
label_2_valid_df = pd.DataFrame(y_valid_pred_conf, columns=['label_2'])
label_2_test_df = pd.DataFrame(y_test_pred_conf, columns=['label_2'])

label_2_valid_df.to_csv('layer_11_label_2_valid.csv', index=False)
label_2_test_df.to_csv('layer_11_label_2_test.csv', index=False)

In [12]:
classifier3 = svm.SVC(kernel='rbf', random_state = 42, C= 10, gamma= 'scale')
classifier3.fit(data_dict['label_3']['X_train'], data_dict['label_3']['Y_train'])

y_valid_pred_conf = classifier3.predict(data_dict['label_3']['X_valid'])
y_test_pred_conf = classifier3.predict(data_dict['X_test'])
label_3_valid_df = pd.DataFrame(y_valid_pred_conf, columns=['label_3'])
label_3_test_df = pd.DataFrame(y_test_pred_conf, columns=['label_3'])

label_3_valid_df.to_csv('layer_11_label_3_valid.csv', index=False)
label_3_test_df.to_csv('layer_11_label_3_test.csv', index=False)

In [13]:
classifier4 = svm.SVC(kernel='rbf', random_state = 42, C= 100, gamma= 'scale')
classifier4.fit(data_dict['label_4']['X_train'], data_dict['label_4']['Y_train'])

y_valid_pred_conf = classifier4.predict(data_dict['label_4']['X_valid'])
y_test_pred_conf = classifier4.predict(data_dict['X_test'])
label_4_valid_df = pd.DataFrame(y_valid_pred_conf, columns=['label_4'])
label_4_test_df = pd.DataFrame(y_test_pred_conf, columns=['label_4'])

label_4_valid_df.to_csv('layer_11_label_4_valid.csv', index=False)
label_4_test_df.to_csv('layer_11_label_4_test.csv', index=False)

In [10]:
df1 = pd.read_csv('layer_8_label_4_test.csv')
df2 = pd.read_csv('layer_11_label_4_test.csv')

# Extract the two columns you want from each DataFrame
# Replace 'column_name_1' and 'column_name_2' with the actual column names from df1
# Replace 'column_name_3' and 'column_name_4' with the actual column names from df2
df1_subset = df1[['layer8label_4']]
df2_subset = df2[['layer11label_4']]

# Concatenate the two DataFrames horizontally to create a new DataFrame with two columns
result = pd.concat([df1_subset, df2_subset], axis=1)

# Save the result to a new CSV file
result.to_csv("./Predictions/Test/8_11_test_label_4.csv", index=False)
